Dataset notebook: 
- extend missense variant tables with charge "DCharge" and hydrophobicity changes "DChargeHP"
- binding and active site locations from Uniprot

Pipeline notebook:
- variant median ddgs
- 

Statscores:
- thresholds and local minima (to be applied to all variants)

TO DO:
- Statscore notebook: variant prediction and peak
- Prody: hinge sites
- Calculateproperties - residue level:
  - MoreRONN: residue disorder score
  - Residue distance to hinge sites, binding sites, active sites
  - Surface area


In [16]:
import json
import pandas as pd
import numpy as np

In [2]:
gene = "PTPN11"

In [ ]:
# Load Uniprot features

#### Metapredictors

In [5]:
with open(f"{gene}_thresholds.json", "r") as f:
    thresholds = json.load(f)

with open(f"{gene}_localmins.json", "r") as f:
    localmins = json.load(f)


In [7]:
def apply_threshold_prediction(df, score, threshold):
    """
    Adds predicted columns using given threshold.
    """

    pred_col = f"Predicted_{score}"

    df[pred_col] = np.where(
        df[score] >= threshold,
        "Pathogenic",
        np.where(df[score].notna(), "Benign", None)
    )

    df[f"{pred_col}_numeric"] = df[pred_col].map({'Benign': 0, 'Pathogenic': 1})

    return df


In [8]:
def add_peak_class(df, localmins_dict):
    df = df.copy()  # avoid modifying original
    
    for score_name, mins in localmins_dict.items():
        col = score_name
        new_col = f"{score_name}_peaks"

        # Ensure minima sorted
        m1, m2 = sorted(mins)

        # Assign bins:
        df[new_col] = np.select(
            [
                df[col] < m1,
                (df[col] >= m1) & (df[col] <= m2), 
                df[col] > m2
            ],
            [0, 0.5, 1],
            default=np.nan
        )

    return df



In [12]:
revel_bayesdel = pd.read_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_REVEL_BayesDel.csv")


In [19]:
revel_bayesdel_labeled = revel_bayesdel.copy()
scores = ["REVEL", "BayesDel_noAF", "BayesDel_addAF"]

for score in scores:
    apply_threshold_prediction(revel_bayesdel, score, thresholds[score])


In [23]:
revel_bayesdel_labeled = add_peak_class(revel_bayesdel, localmins)
revel_bayesdel_labeled = add_peak_class(revel_bayesdel, localmins)


revel_bayesdel_labeled.to_csv(r"\\rdp.arc.ucl.ac.uk\ritd-ag-project-rd025c-bhall50\ModellingRASopathies RDSS\PTPN11_REVEL_BayesDel_labeled.csv", index=False)


In [24]:
revel_bayesdel_labeled

,chr,hg19_pos,grch38_pos,ref,alt,aaref,aaalt,REVEL,Ensembl_transcriptid,BayesDel_noAF,...,protein_pos,Predicted_REVEL,Predicted_REVEL_numeric,Predicted_BayesDel_noAF,Predicted_BayesDel_noAF_numeric,Predicted_BayesDel_addAF,Predicted_BayesDel_addAF_numeric,REVEL_peaks,BayesDel_noAF_peaks,BayesDel_addAF_peaks
0,12,112856917,112419113,T,A,M,K,0.294,ENST00000392597;ENST00000351677;ENST00000392596,0.660000,...,1.0,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
1,12,112856917,112419113,T,C,M,T,0.315,ENST00000392597;ENST00000351677;ENST00000392596,0.660000,...,1.0,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
2,12,112856917,112419113,T,G,M,R,0.202,ENST00000392597;ENST00000351677;ENST00000392596,0.660000,...,1.0,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
3,12,112856918,112419114,G,A,M,I,0.222,ENST00000392597;ENST00000351677;ENST00000392596,0.660000,...,1.0,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
4,12,112856918,112419114,G,C,M,I,0.222,ENST00000392597;ENST00000351677;ENST00000392596,0.660000,...,1.0,Benign,0,Pathogenic,1,Pathogenic,1,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,12,112942557,112504753,A,G,S,G,0.177,ENST00000351677,-0.245014,...,591.0,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0
3942,12,112942557,112504753,A,T,S,C,0.223,ENST00000351677,-0.191480,...,591.0,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0
3943,12,112942558,112504754,G,A,S,N,0.192,ENST00000351677,-0.403129,...,591.0,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0
3944,12,112942558,112504754,G,C,S,T,0.251,ENST00000351677,-0.306398,...,591.0,Benign,0,Benign,0,Benign,0,0.0,0.0,0.0
